In [ ]:
# Import Libraries

import os
import time
import dronekit
import subprocess
import socket
import math

In [ ]:
# Open and load drone swarm file

# Get paths
home_directory = os.path.expanduser("~")
world_file_path = 'Integra_UAV/ardupilot/libraries/SITL/examples/Webots_Python/worlds'
world_file_name = 'uav_swarm_(square_16_drones).wbt'

# Path to your .wbt file
wbt_file_path = f'{home_directory}/{world_file_path}/{world_file_name}'

print(wbt_file_path)

# Check if the file exists
if os.path.exists(wbt_file_path):
    try:
        # Launch Webots with the .wbt file
        subprocess.Popen(['webots', wbt_file_path])
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print(f"File not found: {wbt_file_path}")

In [ ]:
DRONE_COUNT = 16

# List of supported modes and their corresponding messages
MODE_MESSAGES = {
    'land': "Landing vehicle #{i}...",
    'guided': "Changing to GUIDED mode for vehicle #{i}...",
    'loiter': "Changing to LOITER mode for vehicle #{i}...",
    'rtl': "Returning to launch for vehicle #{i}...",
    'circle': "Switching to CIRCLE mode for vehicle #{i}...",
    'stabilize': "Switching to STABILIZE mode for vehicle #{i}...",
    'auto': "Switching to AUTO mode for vehicle #{i}...",
    'alt_hold': "Switching to ALT_HOLD mode for vehicle #{i}...",
    'poshold': "Switching to POSHOLD mode for vehicle #{i}...",
    'acro': "Switching to ACRO mode for vehicle #{i}..."
}

# Circle parameters
CIRCLE_RADIUS = 300  # 3 meters
CIRCLE_RATE = 20     # 20 degrees/sec
CIRCLE_YAW_BEHAVIOR = 0  # Face center

# Formation Center
formation_center = (0, 0)  # Adjust as needed

target_location_list = []

In [ ]:
# Setup Ardupilot SITL Connections 

# Define the correct full path to sim_vehicle.py
SIM_VEHICLE_PATH = f"{home_directory}/Integra_UAV/ardupilot/Tools/autotest/sim_vehicle.py"

# Base command
base_cmd = (
    "python {sim_vehicle}"
    " -v ArduCopter -w --model webots-python"
    " --add-param-file={home_directory}Integra_UAV/ardupilot/libraries/SITL/examples/Webots_Python/params/{param_file}"
    " -I{instanceId} --sysid={SYSID} --out=udp:127.0.0.1:{udp_port}"
)

# Create Specific UDP ports
udp_ports = []

for i in range(DRONE_COUNT):
    newPort = 14551
    newPort = newPort + (i * 10)
    udp_ports.append(newPort)


# Function to check if a port is open (indicating the drone is initialized)
def is_port_open(host, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.settimeout(1)  # Set a timeout for the connection attempt
        try:
            sock.connect((host, port))  # Try to connect to the given host and port
            return True  # Port is open
        except (socket.timeout, socket.error):
            return False  # Port is not open

host = "127.0.0.1"  # Localhost
first_instance_port = 5760
isWait = True

# Launch drones sequentially
for i in range(DRONE_COUNT):
    drone_name = f"iris_drone{i+1}"
    param_file = "iris.parm"
    SYSID = i + 1

    command = base_cmd.format(
        sim_vehicle=SIM_VEHICLE_PATH,
        param_file=param_file,
        instanceId=i,
        SYSID=SYSID,
        udp_port=udp_ports[i]
    )

    print(command)

    # Launch the first drone
    print(f"Starting {drone_name}: #{i+1}")
    process = subprocess.Popen(['xterm', '-e', command])

    while isWait: 

        if is_port_open(host, first_instance_port):
            print(f'Done Build')
            isWait = False
            break
        else:
            print(f'Building in Progress, Please wait')

        time.sleep(2)

    time.sleep(2)
    print(f"{drone_name} has been successfully launched in a new terminal.\n")

print("All drones launched successfully.")

In [ ]:
# Connect Drones

vehicles = []

# Connect to each drone
for i in range(DRONE_COUNT):
    connection_str = f"127.0.0.1:{udp_ports[i]}"
    print(f"Connecting to vehicle #{i+1} on {connection_str}")
    vehicle = dronekit.connect(ip=connection_str, wait_ready=True)
    vehicles.append(vehicle)

print(f"\nConnected to {len(vehicles)} vehicles.\n")

In [ ]:
# Change drones mode 
# land,guided,loiter,rtl,circle,stabilize,auto,alt_hold,poshold,acro

def change_mode(change_mode):
    mode = change_mode.lower()
    
    if mode in MODE_MESSAGES:
        for i, vehicle in enumerate(vehicles):
            print(MODE_MESSAGES[mode].format(i=i))
            vehicle.mode = dronekit.VehicleMode(mode.upper())  # Ensure mode is in uppercase
    else:
        print(f"Error: '{change_mode}' is not a supported flight mode.")


In [ ]:
# Function to Arm drones

def arm_all_vehicles(vehicles):
    """
    Sets GUIDED mode and arms each vehicle. Waits until all are armed.
    """
    print("Setting all vehicles to GUIDED mode...")
    change_mode('GUIDED')

    print("Arming all vehicles...")
    for i, v in enumerate(vehicles):
        v.armed = True

    while True:
        states = [v.armed for v in vehicles]
        print("Armed states:", states)
        if all(states):
            print("All vehicles are armed!")
            break
        time.sleep(1)

In [ ]:
# Function to takeoff drones

def takeoff_all_vehicles(vehicles, altitude):
    """
    Commands each vehicle to take off to the same altitude.
    Waits until all have reached 95% of the target altitude.
    """
    # print(f"Taking off all vehicles to {altitude} meters...")

    # 1) Issue takeoff commands:
    for i, v in enumerate(vehicles):
        print(f"Vehicle #{i} takeoff to {altitude} m")
        v.simple_takeoff(altitude)

    # 2) Wait until each drone reaches *its own* target altitude
    while True:
        # Get current altitudes
        altitudes = [v.location.global_relative_frame.alt for v in vehicles]
        print("Altitudes:", altitudes)

        # Check if each drone is at least 95% of its target altitude
        all_reached = True
        for i, curr_alt in enumerate(altitudes):
            if curr_alt < 0.95 * altitude:
                all_reached = False
                break

        if all_reached:
            print("All vehicles reached their respective altitudes!")
            break

        time.sleep(1)

In [ ]:
def change_altitude_all(vehicles, target_altitude):



    for i, vehicle in enumerate(vehicles):
        print(f"Changing altitude of Vehicle #{i+1} to {target_altitude} meters...")
        current_location = vehicle.location.global_relative_frame
        target_location = dronekit.LocationGlobalRelative(current_location.lat, current_location.lon, target_altitude)
        target_location_list.append(target_location)

    for i, vehicle in enumerate(vehicles):
        vehicle.simple_goto(target_location_list[i])

    # Monitor all drones
    while True:
        all_reached = True
        for i, vehicle in enumerate(vehicles):
            current_alt = vehicle.location.global_relative_frame.alt
            print(f"Vehicle #{i+1} Altitude: {current_alt:.2f} meters")

            if abs(current_alt - target_altitude) > 0.5:
                all_reached = False

        if all_reached:
            print("All drones reached the target altitude.")
            break

        time.sleep(1)

In [ ]:
# Arm Drones
arm_all_vehicles(vehicles)

In [ ]:
# Takeoff Drones with initial 3 meters altitude
takeoff_all_vehicles(vehicles, 15)

In [ ]:
# Land Drones
change_mode('LAND')

In [ ]:
# Return to launch
change_mode('RTL')

In [ ]:
# Change altitude
new_altitude = 20

change_altitude_all(vehicles, new_altitude)

In [ ]:
for i, vehicle in enumerate(vehicles):
    my_lat =vehicle.location.global_relative_frame.lat
    my_lon =vehicle.location.global_relative_frame.lon

    print(f'drone{i}: {my_lat}, {my_lon} \n')



In [ ]:
# Compute Distance between two drones based on lat & lon

R = 6378137.0  # Earth radius in meters (WGS84)

# Drone 0: lat1, lon1
lat1, lon1 = -35.3632621, 149.1651934
# Drone 1: lat2, lon2
lat2, lon2 = -35.3633158, 149.1652594

# Convert degrees to radians
phi1 = math.radians(lat1)
phi2 = math.radians(lat2)
lambda1 = math.radians(lon1)
lambda2 = math.radians(lon2)

delta_phi = phi2 - phi1  # Should be 0 in this case
delta_lambda = lambda2 - lambda1

# Haversine formula
a = (math.sin(delta_phi / 2))**2 + math.cos(phi1)*math.cos(phi2)*(math.sin(delta_lambda / 2))**2
c = 2 * math.asin(math.sqrt(a))
distance = R * c

print(f"Distance: {distance:.3f} meters")
